In [1]:
import os
import datetime
import time

import ipywidgets as widgets
from IPython.display import display

import IPython
import IPython.display
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import clear_output

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

from math import sqrt
from numpy import concatenate
import numpy as np
import math

import pandas as pd
from pandas import DataFrame
from pandas import concat

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

# import tensorflow 
# import tensorflow as tf
# from tensorflow import keras

import platform
# print(platform.python_version())
# print(tf.version.VERSION)
# print(np.__version__)



In [2]:
# File name and Path
cwd = os.getcwd()
path_adress = "/Import_Dataset/"

# ข้อมูล พิกัดสถานที่เเละรายชื้ออ้างอิงสถานที่ตรวจวัด
file_name_st = 'Data_lat_long_Rice research Center'
csv_file_st = cwd + path_adress + file_name_st + '.csv'

df_st = pd.read_csv(csv_file_st)
print("All low RiceCenter {} station" .format(df_st.shape[0]))

data_list_st_num = list(range(0, len(df_st)))
data_list_st_name = []
data_list_st_name = df_st['nameEng'].values.tolist()
data_list_st_lat = df_st['Latitude'].values.tolist()
data_list_st_long = df_st['Longitude'].values.tolist()

All low RiceCenter 34 station


In [29]:
def clicked(b):
    output_df_train.clear_output()
    output_df_train.clear_output()
    with output_df_train:
        display(creat_dataset(0,'2015','2017',0))
        # print("Button clicked.")
        # # frames_train = creat_dataset(2,dropdown_year_train1.value,dropdown_year_train2.value,1)
        # # del frames_train
        # frames_train = creat_dataset(0,'2015','2016',0)
        

In [4]:
ALL = 'ALL'

def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [5]:
def dropdown_year_eventhandler(change):
    output_df_st.clear_output()
    with output_df_st:  
        if (change.new == ALL):
            display(df_st)
        else:
            display(df_st[df_st.nameEng == change.new])

In [6]:
def creat_avrdata(frames_sma,rolling_num):
    
    h_name = list(frames_sma)
    frames_sma['mirid bug'] = frames_sma['mirid bug'].rolling(rolling_num, min_periods=1).sum()
    frames_sma['bph'] = frames_sma['bph'].rolling(rolling_num, min_periods=1).sum()
    for i in range(7,14):
        frames_sma[h_name[i]] = frames_sma[h_name[i]].rolling(rolling_num, min_periods=1).mean()
#        print(header_name[i])

    #defining rate
    our_rate = rolling_num
    #apply the rate
    constant_subset = frames_sma[::our_rate] 
        
    frames = constant_subset
    return frames


In [7]:
def creat_dataset(st_BPH = 0, y_1='2015', y_2='2019', mode=1):
    for i in range(len(data_list_st_num)):
        st = i
        file_name = df_st['nameEng'][data_list_st_num[st]]
        name_input = file_name
        name_locals = 'stN_' + file_name

        # File name and Path
        csv_file = cwd + path_adress + name_input + '.csv'
        # index_col=0 , index_col=None
        dataset = pd.read_csv(csv_file, header=0,
                              index_col=0, encoding="TIS-620")
        # dataset = dataset.drop(['address'], axis=1)
        dataset = dataset.drop(['year'], axis=1)
        dataset = dataset.drop(['dew'], axis=1)
        # dataset = dataset.drop(['latitude'], axis=1)
        # dataset = dataset.drop(['longitude'], axis=1)
        # dataset.columns
        locals()[name_locals] =dataset
        # locals()[name_locals] = creat_avrdata(dataset,1)
        # print(f'Dataframe name_station: {st+1 , name_input}')
        print('wait......')
        del dataset
        clear_output(wait=True)

    date_start = y_1 + '-01' + '-01'
    date_stop = y_2 + '-12' + '-31'

#---------------------------------------------------------------------------------------------------------------------------------------------------#
    if mode == 0:
        file_name = df_st['nameEng'][data_list_st_num[st_BPH]]
        # file_name = st_BPH
        locals_input = 'stN_' + file_name
        print(locals_input)
        dataset_st = locals()[locals_input].loc[date_start:date_stop]
        frames_st = dataset_st
    else:
        m = 0
        for j in range(len(data_list_st_num)):
            # for j in range(df_st.shape[0]):
            # file_name = df_st['nameEng'][j]
            file_name = df_st['nameEng'][data_list_st_num[j]]
            locals_input = 'stN_' + file_name
            print(locals_input)
            dataset_st = locals()[locals_input].loc[date_start:date_stop]
            clear_output(wait=True)
            if m == 0:
                frames_st = dataset_st
                m = m+1
                print(m)
            else:
                frames_st = [frames_st, dataset_st]
                frames_st = pd.concat(frames_st)
#---------------------------------------------------------------------------------------------------------------------------------------------------#
    # dataset
    
    frames = frames_st
    return frames


In [28]:


output_df_st = widgets.Output()

dropdown_name_st = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_st.nameEng),description='station :')
dropdown_sampling_data = widgets.Dropdown(options = ['All DAY','3-DAY','7-DAY'],valure = 'All-DAY',description='Sampling :')
dropdown_year_train1 = widgets.Dropdown(options = ['2015','2016','2017','2018','2019','2020'],description='Year-train :')
dropdown_year_train2 = widgets.Dropdown(options = ['2015','2016','2017','2018','2019','2020'],value = '2017')
dropdown_year_val1 = widgets.Dropdown(options = ['2015','2016','2017','2018','2019','2020'],value = '2018',description='Year-val :')
dropdown_year_val2 = widgets.Dropdown(options = ['2015','2016','2017','2018','2019','2020'],value = '2018')
dropdown_year_test1 = widgets.Dropdown(options = ['2015','2016','2017','2018','2019','2020'],value = '2019',description='Year-test :')
dropdown_year_test2 = widgets.Dropdown(options = ['2015','2016','2017','2018','2019','2020'],value = '2019')

button_download = widgets.Button(description='load data',disabled=False,
                                button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                                tooltip='Click me',
                                 icon='check' # (FontAwesome names without the `fa-` prefix) 
                                )
button_download.on_click(clicked)

output_df_train = widgets.Output()

dropdown_name_st.observe(dropdown_year_eventhandler, names='value')




# display(dropdown_name_st,dropdown_sampling_data,dropdown_year_train1,dropdown_year_train2,dropdown_year_val1,
#         dropdown_year_val2,dropdown_year_test, button_download,output_download)

input_widgets_row1 = widgets.HBox([dropdown_name_st, dropdown_year_train1, dropdown_year_train2,dropdown_year_val1,dropdown_year_val2,dropdown_year_test1,dropdown_year_test2])
input_widgets_row2 = widgets.HBox([dropdown_sampling_data])

tab = widgets.Tab([output_df_st,output_df_train])
tab.set_title(0, 'Dataset station')
tab.set_title(1, 'Dataset train')

dashboard = widgets.VBox([input_widgets_row1,input_widgets_row2,button_download,tab])
display(dashboard)

In [ ]:
st = 1
# st_num = 1
file_name = df_st['nameEng'][data_list_st_num[st]]
name_input = file_name
name_locals = 'stN_' + file_name
print(name_locals)

# File name and Path
csv_file = cwd + path_adress + name_input + '.csv'
# index_col=0 , index_col=None
dataset = pd.read_csv(csv_file, header=0,
                      index_col=0, encoding="TIS-620")

header_name = list(dataset)
# print(header_name)

data_list_header_num = list(range(0, len(header_name)))
# print(data_list_header_num)

data_list_drop = [0,4,5]

for i in range(len(data_list_drop)):
        dataset = dataset.drop([header_name[data_list_drop[i]]], axis=1)

dataset.head()


In [ ]:
# dataset
frames_train = creat_avrdata(dataset,7)
frames_train.head()
# len(frames_train)

In [ ]:
# frames_train.head()
dataset.head()

In [ ]:
# df_temperature = dataset['temp']
df_temperature = pd.DataFrame(dataset['bph'].loc['2015-01-01':'2015-01-31']) 

# df_temperature = 
# the simple moving average over a period of 10 years
# df_temperature['7D-SMA'] = df_temperature.bph.rolling(7, min_periods=1).mean()
# df_temperature['3D-SMA'] = df_temperature.bph.rolling(3, min_periods=1).mean()
df_temperature['7D-SMA'] = df_temperature.bph.rolling(7, min_periods=1).sum()
df_temperature['3D-SMA'] = df_temperature.bph.rolling(3, min_periods=1).sum()
# df_temperature

In [ ]:
# colors for the line plot
colors = ['green', 'red', 'purple']

# line plot - the yearly average air temperature in Barcelona
df_temperature.plot(color=colors, linewidth=3, figsize=(12,6))

# modify ticks size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(labels =['Data Input', '10-years SMA', '20-years SMA'], fontsize=14)

# title and labels
plt.title('Data average ', fontsize=18)
plt.xlabel('date-time', fontsize=16)
plt.ylabel('Temperature [°C]', fontsize=16)

In [ ]:
#defining rate
our_rate = 6 
#apply the rate
constant_subset = df_temperature[::our_rate] 
#data
constant_subset

In [ ]:
# frames_train
our_rate = 6 

#conditonal sampling
our_condition = df['latitude'] == 15.198

#Retirive the index
index = our_condition[our_condition == True].index
 
#sample based on condition 
conditional_subset = df[our_condition][::our_rate] 
 
#output 
df2 = conditional_subset.set_index()
df2

In [ ]:
our_condition

In [ ]:
#defining rate
our_rate = 6 
#apply the rate
constant_subset = df_temperature[::our_rate] 
#data
constant_subset